In [1]:
# Step 1: Import Libraries
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 2: Load Data
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Step 3: Create User-Item Matrix
user_movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Step 4: Apply Matrix Factorization (SVD)
svd = TruncatedSVD(n_components=20, random_state=42)
matrix_reduced = svd.fit_transform(user_movie_matrix)

# Step 5: Compute Similarity Between Users
similarity = cosine_similarity(matrix_reduced)

# Step 6: Get Recommendations for a User
def recommend_movies(user_id, user_movie_matrix, movies, similarity_matrix, n=5):
    user_index = user_id - 1  # Because userId starts from 1
    similar_users = list(enumerate(similarity_matrix[user_index]))
    similar_users = sorted(similar_users, key=lambda x: x[1], reverse=True)[1:11]

    weighted_ratings = np.zeros(user_movie_matrix.shape[1])
    similarity_sum = np.zeros(user_movie_matrix.shape[1])

    for other_index, score in similar_users:
        weighted_ratings += user_movie_matrix.iloc[other_index].values * score
        similarity_sum += (user_movie_matrix.iloc[other_index].values > 0) * score

    scores = weighted_ratings / np.where(similarity_sum == 0, 1, similarity_sum)

    already_rated = user_movie_matrix.iloc[user_index].values > 0
    scores[already_rated] = 0

    top_indices = np.argsort(scores)[-n:][::-1]
    movie_ids = user_movie_matrix.columns[top_indices]

    return movies[movies['movieId'].isin(movie_ids)][['movieId', 'title']]

# Step 7: Recommend Movies for User 1
recommendations = recommend_movies(1, user_movie_matrix, movies, similarity)
print("Top Movie Recommendations for User 1:")
print(recommendations)


Top Movie Recommendations for User 1:
      movieId                                        title
695       913                   Maltese Falcon, The (1941)
809      1059  William Shakespeare's Romeo + Juliet (1996)
1711     2300                        Producers, The (1968)
2308     3060                      Commitments, The (1991)
2562     3429                     Creature Comforts (1989)
